# ETHIC - Primavera 2024

##  Selección de caso

* 1) Caso Amanda

In [1]:
from utils.extract_data import process_data
from utils.extract_data import read_data

caso = process_data()
df1, df2 = read_data(caso)

Casos disponibles:
1. Amanda
Caso seleccionado: Amanda
Archivo: Procesamient_2023_CD1201_2.xlsx, Año: 2023, Sección: 2
Archivo: Procesamient_2023_CD1201_3.xlsx, Año: 2023, Sección: 3
Archivo: Procesamient_2023_CD1201_4.xlsx, Año: 2023, Sección: 4
Archivo: Procesamient_2023_CD1201_5.xlsx, Año: 2023, Sección: 5
Archivo: Procesamient_2023_CD1201_7.xlsx, Año: 2023, Sección: 7
Archivo: Procesamient_2023_CD1201_8.xlsx, Año: 2023, Sección: 8
Archivo: Procesamient_2023_CD1201_9.xlsx, Año: 2023, Sección: 9
Archivo: Procesamient_2024_CD1100_2.xlsx, Año: 2024, Sección: 2
Archivo: Procesamient_2024_CD1100_4.xlsx, Año: 2024, Sección: 4
Archivo: Procesamient_2024_CD1100_6.xlsx, Año: 2024, Sección: 6
Archivo: Procesamient_2024_CD1100_7.xlsx, Año: 2024, Sección: 7
Archivo: Procesamient_2024_CD1100_8.xlsx, Año: 2024, Sección: 8

Carpeta ya existe: processed_data/Amanda
Datos procesados guardados en: processed_data/Amanda/answers_by_secc_Amanda.csv

Datos leídos de: processed_data/Amanda/answers_by_secc

## Analisis Gramatical

Se debe ejecutar la celda de apply_dependency_analisis solo si se desea realizar un nuevo analisis. De lo contrario basta con ejecutar la celda de graficar.

In [4]:
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.dependency_tree import *

# Función para analizar oraciones y generar su árbol de dependencias
def analizar_dependencias(texto):
    # Cargar el modelo de SpaCy en español
    nlp = spacy.load("es_core_news_md")
    
    # Procesar el texto con SpaCy
    doc = nlp(texto)
    
    # Inicializar contador de estructuras subordinadas
    total_subordinadas = 0
    
    # Recorrer cada oración en el documento
    for sentencia in doc.sents:
        # Contar oraciones subordinadas en esta oración
        oraciones_subordinadas = [token for token in sentencia if token.dep_ in ("acl", "advcl", "csubj", "ccomp", "xcomp")]
        
        # Sumar al total
        total_subordinadas += len(oraciones_subordinadas)
    
    # Devolver el número total de estructuras subordinadas encontradas
    return total_subordinadas

# Función optimizada para aplicar dependencias solo una vez por grupo, año, y sección
def aplicar_dependencias_grupal(df, caso, diferencial, col_ind1, col_grup, col_ind2):
    # Aplicar analizar_dependencias directamente a col_ind1 y col_ind2 (en todas las filas)
    df['Ind1_d'] = df[col_ind1].apply(lambda x: analizar_dependencias(str(x)))
    df['Ind2_d'] = df[col_ind2].apply(lambda x: analizar_dependencias(str(x)))
    
    # Agrupar por 'Grup', 'agno', 'seccion' y obtener filas únicas
    grouped = df[['Grup', 'agno', 'seccion', col_grup]].drop_duplicates(subset=['Grup', 'agno', 'seccion'])
    
    # Aplicar analizar_dependencias solo una vez para 'col_grup' en las combinaciones únicas
    grouped['Grup_d'] = grouped[col_grup].apply(lambda x: analizar_dependencias(str(x)))
    
    # Unir el resultado de 'Grup_d' al dataframe original, replicando para cada combinación
    df = df.merge(grouped[['Grup', 'agno', 'seccion', 'Grup_d']], 
                  on=['Grup', 'agno', 'seccion'], 
                  how='left')
    
    # Guardar el dataframe
    df.to_csv(f'resultados/{caso}/df{str(diferencial)}_dependencias.csv', index=False)
    
    return df

In [6]:
# Aplicar la función optimizada a ambos DataFrames
df_diferencial_1 = aplicar_dependencias_grupal(df1, caso, 1, 'Comentario - Ind1 - Diferencial 1', 
                                               'Comentario - Grup - Diferencial 1', 
                                               'Comentario - Ind2 - Diferencial 1')

df_diferencial_2 = aplicar_dependencias_grupal(df2, caso, 2, 'Comentario - Ind1 - Diferencial 2', 
                                               'Comentario - Grup - Diferencial 2', 
                                               'Comentario - Ind2 - Diferencial 2')

: 

### Graficos

## Analisis de Topicos BERT

In [1]:
import re
import spacy
import pandas as pd
from nltk.stem import SnowballStemmer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

ModuleNotFoundError: No module named 'nltk'

## Analisis de Topics Eticos